In [ ]:
!pip install pytube opencv-python-headless deepface scikit-learn pandas tqdm ffmpeg-python

['Collecting pytube',
 '  Downloading pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)',
 'Requirement already satisfied: opencv-python-headless in /usr/local/lib/python3.10/dist-packages (4.10.0.84)',
 'Collecting deepface',
 '  Downloading deepface-0.0.93-py3-none-any.whl.metadata (30 kB)',
 'Requirement already satisfied: scikit-learn in /usr/local/lib/python3.10/dist-packages (1.5.2)',
 'Requirement already satisfied: pandas in /usr/local/lib/python3.10/dist-packages (2.2.2)',
 'Requirement already satisfied: tqdm in /usr/local/lib/python3.10/dist-packages (4.66.6)',
 'Collecting ffmpeg-python',
 '  Downloading ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)',
 'Requirement already satisfied: numpy>=1.21.2 in /usr/local/lib/python3.10/dist-packages (from opencv-python-headless) (1.26.4)',
 'Requirement already satisfied: requests>=2.27.1 in /usr/local/lib/python3.10/dist-packages (from deepface) (2.32.3)',
 'Requirement already satisfied: gdown>=3.10.1 in /usr/local/lib/pyth

In [ ]:
import pandas as pd

sheet_url = "https://docs.google.com/spreadsheets/d/1Bj4-sd6362GWrFZOPcND3fFo0oroO1pfhkpMJh8iIE4/edit?usp=sharing"
sheet_url = sheet_url.replace('/edit?usp=sharing', '/export?format=csv')
df = pd.read_csv(sheet_url)

print(df.head())

   Performance                                          Video URL
0       1.1060  https://fgimagestorage.blob.core.windows.net/f...
1       2.2447  https://fgimagestorage.blob.core.windows.net/f...
2       2.0126  https://fgimagestorage.blob.core.windows.net/f...
3       1.7708  https://fgimagestorage.blob.core.windows.net/f...
4       0.6293  https://fgimagestorage.blob.core.windows.net/f...


In [ ]:
df = df.dropna(subset=['Video URL', 'Performance'])
df['Performance'] = pd.to_numeric(df['Performance'], errors='coerce')

In [ ]:
import requests

def download_video(link, output_path):
    try:
        response = requests.get(link, stream=True, timeout=10)
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    f.write(chunk)
            print(f"Downloaded: {output_path}")
        else:
            print(f"Failed to download {link}: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {link}: {e}")

for i, link in enumerate(df['Video URL']):
    output_file = f'videos/video_{i}.mp4'
    download_video(link, output_file)

Downloaded: videos/video_0.mp4
Downloaded: videos/video_1.mp4
Downloaded: videos/video_2.mp4
Downloaded: videos/video_3.mp4
Downloaded: videos/video_4.mp4
Downloaded: videos/video_5.mp4
Downloaded: videos/video_6.mp4
Downloaded: videos/video_7.mp4
Downloaded: videos/video_8.mp4
Downloaded: videos/video_9.mp4
Downloaded: videos/video_10.mp4
Downloaded: videos/video_11.mp4
Downloaded: videos/video_12.mp4
Downloaded: videos/video_13.mp4
Downloaded: videos/video_14.mp4
Downloaded: videos/video_15.mp4
Downloaded: videos/video_16.mp4
Downloaded: videos/video_17.mp4
Downloaded: videos/video_18.mp4
Downloaded: videos/video_19.mp4
Downloaded: videos/video_20.mp4
Downloaded: videos/video_21.mp4
Downloaded: videos/video_22.mp4
Downloaded: videos/video_23.mp4
Downloaded: videos/video_24.mp4
Downloaded: videos/video_25.mp4
Downloaded: videos/video_26.mp4
Downloaded: videos/video_27.mp4
Downloaded: videos/video_28.mp4
Downloaded: videos/video_29.mp4
Downloaded: videos/video_30.mp4
Downloaded: videos

In [ ]:
!pip install retina-face

In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
from retinaface import RetinaFace
import cv2
import os
from tqdm import tqdm

def extract_faces_retina(video_path, output_dir="faces"):
    """
    Extracts faces from a video using RetinaFace and saves them as images.

    Parameters:
    - video_path: Path to the input video file.
    - output_dir: Directory to save the extracted faces.
    """
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    face_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        faces = RetinaFace.detect_faces(frame)

        for face_key in faces.keys():
            face_data = faces[face_key]
            x, y, x2, y2 = face_data['facial_area']
            x, y, x2, y2 = max(0, x), max(0, y), x2, y2
            face = frame[y:y2, x:x2]

            if face.shape[0] > 50 and face.shape[1] > 50:
                face_path = os.path.join(output_dir, f"{os.path.basename(video_path)}_frame{frame_count}_face{face_count}.jpg")
                cv2.imwrite(face_path, face)
                face_count += 1

        frame_count += 1

    cap.release()
    print(f"Faces from {video_path} saved to {output_dir}.")

def process_all_videos(video_folder="videos", output_folder="faces"):
    """
    Process all video files in a directory to extract faces.

    Parameters:
    - video_folder: Path to the folder containing input videos.
    - output_folder: Path to the folder where faces will be saved.
    """
    video_paths = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith(('.mp4', '.avi', '.mkv'))]
    for video_path in tqdm(video_paths, desc="Processing Videos"):
        video_output_dir = os.path.join(output_folder, os.path.basename(video_path).split('.')[0])
        extract_faces_retina(video_path, video_output_dir)

process_all_videos(video_folder="videos", output_folder="faces")

Processing Videos:   0%|          | 0/268 [00:00<?, ?it/s]

24-12-01 04:50:03 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5

  0%|          | 0.00/119M [00:00<?, ?B/s]
 39%|███▉      | 46.7M/119M [00:00<00:00, 464MB/s]
100%|██████████| 119M/119M [00:00<00:00, 388MB/s] 
Processing Videos:   0%|          | 1/268 [01:35<7:03:47, 95.23s/it]

Faces from videos/video_148.mp4 saved to faces/video_148.


Processing Videos:   1%|          | 2/268 [03:37<8:13:40, 111.36s/it]

...

Faces from videos/video_101.mp4 saved to faces/video_101.


Processing Videos:  37%|███▋      | 100/268 [2:11:08<3:42:20, 79.41s/it]

Faces from videos/video_9.mp4 saved to faces/video_9.


Processing Videos:  37%|███▋      | 100/268 [2:11:23<3:40:43, 78.83s/it]


KeyboardInterrupt: 

In [ ]:
!pip install deepface

In [ ]:
import os
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from deepface import DeepFace

def get_face_embedding(face_image_path):
    try:
        embedding = DeepFace.represent(face_image_path, model_name="Facenet")[0]['embedding']
        return embedding
    except Exception as e:
        print(f"Error processing {face_image_path}: {e}")
        return None

def find_and_remove_duplicates_in_video_folder(video_folder_path, threshold=0.6):
    embeddings = []
    faces = []

    for face_image in os.listdir(video_folder_path):
        face_image_path = os.path.join(video_folder_path, face_image)

        if face_image_path.endswith(".jpg"):
            embedding = get_face_embedding(face_image_path)
            if embedding is not None:
                embeddings.append((face_image_path, embedding))

    unique_faces = []
    unique_embeddings = []

    for face_image_path, embedding in embeddings:
        is_duplicate = False

        for unique_embedding in unique_embeddings:
            similarity = cosine_similarity([embedding], [unique_embedding])[0][0]
            if similarity > threshold:
                is_duplicate = True
                break

        if not is_duplicate:
            unique_faces.append(face_image_path)
            unique_embeddings.append(embedding)

    for face_image_path in embeddings:
        if face_image_path[0] not in unique_faces:
            os.remove(face_image_path[0])

    print(f"Unique faces in {video_folder_path}: {len(unique_faces)}")

faces_dir = "faces"

for video_folder in os.listdir(faces_dir):
    video_folder_path = os.path.join(faces_dir, video_folder)

    if os.path.isdir(video_folder_path):
        print(f"Processing folder: {video_folder}")
        find_and_remove_duplicates_in_video_folder(video_folder_path, threshold=0.6)

In [ ]:
# import os
# import shutil

# # Path to the faces folder
# faces_folder = "yolov5_face"

# # Remove the folder and all its contents
# if os.path.exists(faces_folder):
#     shutil.rmtree(faces_folder)
#     print(f"Cleared the '{faces_folder}' folder.")
# else:
#     print(f"The '{faces_folder}' folder does not exist.")

# # Recreate an empty faces folder
# os.makedirs(faces_folder, exist_ok=True)
# print(f"Recreated an empty '{faces_folder}' folder.")

Cleared the 'yolov5_face' folder.
Recreated an empty 'yolov5_face' folder.
